# Second Part

## Глава 4 

### Текст и байты

In [5]:
# Как бы мы не хотели, от различий между str и bytes никуда не деться;

# Зато специализированные типы двоичных последовательностей
#     обладают возможностями, которых нет у str!

In [6]:
# В этой главе будут рассмотрены следующие вопросы:

#     - Символы, кодовые позиции и байтовые представления;

#     - Уникальные особенности двоичных последовательностей:

#           - bytes;

#           - bytearray;

#           - memoryview;

#     - Кодеки для Unicode и унаследованных наборов символов;

#     - Как предотвращать и обрабатывать ошибки кодирования;

#     - Рекомендации по работе с текстовыми файлами;

#     - Кодировка по умолчанию и стандартные проблемы ввода-вывода;

#     - Безопасное сравнение Unicode-текстов с нормализацей;

#     - Служебные функции для нормализации, сворачивания регистра
#           и явного удаления диакритических знаков;

#     - Правильная сортировка Unicode-текстов 
#           с помощью модуля locale и библиотеки PyUCA;

#     - Символьные метаданные в базе данных Unicode;

#     - Двухрежимные API для работы с типами str и bytes.

### О символах и не только

##### Строка - последовательность символов

In [1]:
# А что такое символ?

# Например, символ Unicode;
# Поэтому отдельные элементы объекта str - символы Unicode;

# Стандарт Unicode явно разделяет идентификатор символа
#     и конкретное байтовое представление;

# Идентификатор символа - кодовая позиция - число,
#     записанное шестнадцатеричными цифрами (от 4 до 6) с префиксом "U+";

# Кодовая позиция буквы A - U+0041, знака евро - U+20AC.

In [2]:
# Кодировка влияет на то, какими конкретно байтами представляется символ;

# Кодировка - алгоритм преобразования 
#     кодовых позиций в последовательности байтов
#         и наоборот;

# Кодовая позиция буквы А (U+0041) 
#     кодируется одним байтом \x41 в кодировке UTF-8
#         и двумя байтами \x41\x00 в кодировке UTF-16LE;

# Преобразование из кодовых позиций в байты
#     называется кодированием,
#         преобразование из байтов
#             в кодовые позиции - декодированием:

In [8]:
s = 'café'
len(s) # строка из четырех символов Unicode

4

In [9]:
b = s.encode('utf8') # преобразуем str в bytes, пользуясь кодировкой utf-8
b   # литералы bytes начинаются префиксом b

b'caf\xc3\xa9'

In [11]:
len(b) # объект bytes состоит из пяти байтов

5

In [12]:
b.decode('utf-8') # преобразуем bytes обратно в str

'café'

##### Кодирование - в bytes, декодирование - в str

### Все, что нужно знать о байтах

In [14]:
# Существует два основных типа двоичных последовательностей:
#     неизменяемый bytes и изменяемый bytearray;

# Каждый элемент bytes или bytearray - 
#     целое число от 0 до 255;

# Срез двоичной последовательности 
#     всегда является двоичной последовательностью,
#         даже если это срез длины 1:

In [16]:
cafe = bytes('café', encoding='utf_8') # str -> bytes
cafe

b'caf\xc3\xa9'

In [17]:
cafe[0] # Каждый элемент - целое число в диапазоне range(256)

99

In [19]:
cafe[:1] # Срез bytes имеет тип bytes

b'c'

In [20]:
cafe_arr = bytearray(cafe)
cafe_arr

bytearray(b'caf\xc3\xa9')

In [22]:
# Интересно, что my_bytes[0] возвращает int,
#     а my_bites[:1] - объект bytes длины 1;

# Единственный тип последовательности,
#     для которого s[0] == s[:1], - это тип str;

# Поведение типа str является исключением, 
#     для всех остальных последовательностей
#         s[i] возвращает один элемент,
#             а s[i:i+1] - последовательность, 
#                 состоящую из элемента s[i].

##### Для разных байтов применяются различные способы отображения:

In [ ]:
#     - Для байтов из диапазона ASCII выводится сам символ ASCII;

#     - Для байтов - символов табуляции, новой строки,
#           возврата каретки и \ выводятся управляющие последовательности:
#               \t, \n, \r и \\;

#     - Для всех остальных байтов выводится 
#           шестнадцатеричное представление (нулевой байт - \x00).

In [2]:
# И bytes, и bytearray поддерживают все методы str,
#     кроме тех, что относятся к форматированию (format, format_map).

In [3]:
# Объекты bytes и bytearray можно вызвать следующим образом:

#     - Через конструктор с аргументами str и encoding;

#     - Используя итерируемый объект со значениями от 0 до 255;

#     - Используя протокол буфера (объекты bytes,bytearray, memoryview, array).

In [4]:
# Инициализация байтов данными массива:
import array

numbers = array.array('h', [-2, -1, 0, 1, 2]) # 'h' - массив коротких целых
octets = bytes(numbers)
octets

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

#### Структуры и представления областей памяти

In [6]:
# Модуль struct содержит функции 
#     как для разбора упакованных байтов,
#         так и для преобразования кортежей в байты;

# Функции из модуля struct применимы к объектам
#     bytes, bytearray, memoryview.

In [ ]:
# Использование memoryview и struct 
#     для извлечения ширины и высоты из заголовка GIF-изображения:

import struct

fmt = '<3s3sHH' # формат struct: 
#                 < - порядок байтов, s3s3;
#                 3s3s - две последовательности по три байта;
#                 HH - два 16-разрядных целых;
with open('filter.gif', 'rb') as fp:
    img = memoryview(fp.read()) # создается memoryview из файла в памяти...
header = img[:10] # и еще один;
# при этом ни один байт не копируется;
bytes(header) # копируется 10 байтов;

In [ ]:
struct.unpack(fmt, header) # memoryview -> кортеж

In [ ]:
del header # освобождаем память,
del img # занятую memoryview

##### Операция среза memoryview возвращает объект memoryview без копирования байтов

### Базовые кодировщики и декодировщики

##### Кодек - кодировщик-декодировщик, преобразовывающий текст в байты

In [4]:
# Строка, закодированная тремя кодеками, дает разные последовательности байтов:
for codec in ['latin_1', 'utf_8', 'utf_16']:
    print(codec, 'El Niño'.encode(codec), sep='\t')

latin_1	b'El Ni\xf1o'
utf_8	b'El Ni\xc3\xb1o'
utf_16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


### Проблемы кодирования и декодирования

In [1]:
# Сущесвует исключение UnicodeError, но чаще всего ошибка специфична:
#     либо UnicodeDecodeError (двоичная последовательность -> str),
#         либо либо UnicodeEncodeError (str -> в двоичную последовательность).

##### Первое, на что нужно обращать внимание - точный тип исключения.

#### Обработка UnicodeEncodeError

In [3]:
# Кодирование текста в байты: обработка ошибок
city = 'São Paulo'
city.encode('utf_8') # кодировки utf справляются с любой строко

b'S\xc3\xa3o Paulo'

In [4]:
city.encode('utf-16')

b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'

In [6]:
city.encode('iso8859_1') # iso также работает для строки

b'S\xe3o Paulo'

In [11]:
try: 
    city.encode('cp437') # cp437 спотыкается на ã
except UnicodeEncodeError:
    print('We\'ve got UnicodeEncodeError!')

We've got UnicodeEncodeError!


In [12]:
city.encode('cp437', errors='ignore') # пропускаем некодируемые символы

b'So Paulo'

In [13]:
city.encode('cp437', errors='replace') # заменяем некодируемые символы на ?

b'S?o Paulo'

In [14]:
city.encode('cp437', errors='xmlcharrefreplace') # заменяем некод. сим-лы на xml

b'S&#227;o Paulo'

#### Обработка UnicodeDecodeError

In [16]:
# Пример декодирования строки в байты:
octets = b'Montr\xe9al'
octets.decode('cp1252') # 'cp1252' - подмножество 'latin'

'Montréal'

octets.decode('iso8859_7') # кодировка греческого языка 
# неправильно интерпретирует байт '\xe9', но исключение не возбуждается

In [18]:
octets.decode('koi8_r') # кодировка русского также неправильно интерпретирует

'MontrИal'

In [19]:
try:
    octets.decode('utf-8') # исключение возбуждается, тк 'utf-8' не знаком
except UnicodeDecodeError: #     с октетамми '\xe9'
    print('We\'ve got an UnicodeDecodeError!')

We've got an UnicodeDecodeError!


In [20]:
octets.decode('utf-8', errors='replace') # знакомый нам обработчик ошибок

'Montr�al'

#### Как определить кодировку последовательности байтов?

##### Коротко: никак

#### ВОМ: полезный крокозябр

In [24]:
u16 = 'El Nino'.encode('utf_16')
u16

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00n\x00o\x00'

In [25]:
another_u16 = 'Moscow'.encode('utf_16')
another_u16

b'\xff\xfeM\x00o\x00s\x00c\x00o\x00w\x00'

### Обработка текстовых файлов

In [27]:
# Пример с ошибкой!
open('cafe.txt', 'w', encoding='utf_8').write('café')

4

In [28]:
open('cafe.txt').read()

'cafГ©'

### Нормализация Unicode для правильного сравнения

In [32]:
s1 = 'café'
s2 = 'cafe\u0301'

s1, s2

('café', 'café')

In [33]:
len(s1), len(s2)

(4, 5)

In [34]:
s1 == s2

False

In [37]:
from unicodedata import normalize

s1 = 'café'
s2 = 'cafe\u0301'

len(s1), len(s2)

(4, 5)

In [38]:
len(normalize('NFC', s1)), len(normalize('NFC', s2))

(4, 4)

In [39]:
len(normalize('NFD', s1)), len(normalize('NFD', s2))

(5, 5)

In [40]:
normalize('NFC', s1) == normalize('NFC', s2)

True

In [41]:
#### Сворачивание регистра

In [43]:
sen = 'Mom I have a letter: μ'
sen.casefold()

'mom i have a letter: μ'

In [44]:
#### Примеры в действии

In [45]:
# Функция удаления модифицирующих символов:
import unicodedata
import string

def shave_marks(txt):
    """ удаление диакритических знаков """
    norm_txt = unicodedata.normalize('NFD', txt)
    shaved = ''.join(c for f in norm_txt
                    if not unicodedata.combining(c))
    return unicodedata.normalize('NFC', shaved)

In [ ]:
### Сортировка Unicode-текстов